# SQL. Sturtup for reading lovers


**Цель исследования:**

сформулировать ценностное предложение для нового продукта сервиса для чтения книг по подписке.

**Задача исследования:**

проанализировать базу данных. 

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;

- `author_id` — идентификатор автора;

- `title` — название книги;

- `num_pages` — количество страниц;

- `publication_date` — дата публикации книги;

- `publisher_id` — идентификатор издателя.


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;

- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;

- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;

- `book_id` — идентификатор книги;

- `username` — имя пользователя, оставившего оценку;

- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;

- `book_id` — идентификатор книги;

- `username` — имя пользователя, написавшего обзор;

- `text` — текст обзора.

**ШАГ 1. Импорт библиотек и загрузка данных**

In [ ]:
import pandas as pd 
from sqlalchemy import create_engine 

db_config = {'user': '...',  
                     'pwd': 'Sdf4$2...',  
                     'host': 'rc1b....net', 
                     'port': 6432, 
                     'db': 'data-anal...'}  
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                                                    db_config['pwd'], 
                                                                         db_config['host'], 
                                                                                    db_config['port'], 
                                                                                    db_config['db']) 
 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
books = ''' SELECT * 
            FROM books
            LIMIT 5'''
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
authors = ''' SELECT * 
            FROM authors
            LIMIT 5'''
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
publishers = ''' SELECT * 
            FROM publishers
            LIMIT 5'''
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
ratings = ''' SELECT * 
            FROM ratings
            LIMIT 5'''
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
reviews = ''' SELECT * 
            FROM reviews
            LIMIT 5'''
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Данные загружены и прочитаны.

**Количество книг, вышедших после 1 января 2000 года.**

In [ ]:
books_number = '''SELECT 
                      COUNT(book_id) count_of_books
                  FROM 
                      books 
                  WHERE 
                      publication_date > '01-01-2000';
'''
pd.io.sql.read_sql(books_number, con = engine)      

,count_of_books
0,819


После 1 января 2000 года вышло 819 книг.

**Количество обзоров и средняя оценка каждой книги.**

In [ ]:
review_rating = '''SELECT
                       books.book_id AS id,
                       books.title AS title,
                       COUNT(DISTINCT reviews.review_id) AS reviews_cnt,
                       AVG(ratings.rating) AS avg_rating
                   FROM 
                       books
                   LEFT JOIN reviews ON  reviews.book_id = books.book_id
                   LEFT JOIN ratings ON  ratings.book_id = books.book_id
                   GROUP BY
                       books.book_id
                   ORDER BY
                       reviews_cnt DESC;
'''    
    
    
pd.io.sql.read_sql(review_rating, con = engine)    

,id,title,reviews_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


**Издательство, которое выпустило наибольшее число книг толще 50 страниц.**

In [ ]:
top_publisher = '''SELECT
                       books.publisher_id AS publisher_id,
                       publishers.publisher,
                       COUNT(books.book_id) AS books_number
                    FROM
                       books
                    INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
                    WHERE
                       books.num_pages > 50
                    GROUP BY
                       books.publisher_id,
                       publishers.publisher
                    ORDER BY
                       books_number DESC
                    LIMIT 1; 
'''
pd.io.sql.read_sql(top_publisher, con = engine)

,publisher_id,publisher,books_number
0,212,Penguin Books,42


Издательство "Penguin Books"выпустило 42 книги толще 50 страниц.

**Автор с самой высокой средней оценкой книг с учетом только книги с 50 и более оценками.**

In [ ]:
top_author = '''SELECT
                    authors.author AS author,
                    COUNT(rating_id) AS rating_cnt,
                    AVG(rating) as rating_avg
                  FROM 
                     ratings
                  INNER JOIN books ON ratings.book_id = books.book_id 
                  INNER JOIN authors ON authors.author_id = books.author_id
                  GROUP BY
                      authors.author
                  HAVING 
                      COUNT(rating_id) > 50     
                  ORDER BY
                      rating_avg DESC
                  LIMIT 1; 

'''
pd.io.sql.read_sql(top_author, con = engine)

,author,rating_cnt,rating_avg
0,J.K. Rowling/Mary GrandPré,312,4.288462


Дж.К. Роулинг - автор с самой высокой средней оценкой книг.

**Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [ ]:
mean_reviews = '''
        SELECT
            AVG(sub.count)
        FROM
            (SELECT COUNT(*)
            FROM 
                reviews
            WHERE 
                username IN
                (SELECT username
                    FROM 
                        ratings
                    GROUP BY 
                        username
                    HAVING 
                        COUNT(rating) > 50)
            GROUP BY 
                username)
            AS sub;
'''
pd.io.sql.read_sql(mean_reviews, con = engine)        

,avg
0,24.333333


Пользователи, которые поставили больше 50 оценок, в среднем сделали 24,3 обзора.

**ИТОГ ИССЛЕДОВАНИЯ:**

После 1 января 2000 года вышло 819 книг.

Издательство Penguin Books является лидером по количеству выпущенных книг с более чем 50 страницами, а именно издательство выпустило 42 таких книги.

Дж.К. Роулинг является авторомс самой высокой средней оценкой книг. 

Пользователи, поставившие более 50 оценок, в среднем сделали 24,3 обзора.

